In [1]:
#inputs
import os
parent_directory = os.path.abspath('')
root = os.path.abspath(os.path.join(parent_directory, os.pardir))
data_folder = '01-Data'
classifiers_folder = '02-Classifiers'
gan_char_models_folder = 'models_char_gan'
model_digits_letters_name = 'class_char_model_{}.h5'
mixed_models_folder = 'models_mixed'
model_symbols_letters_name = 'model_0symbol_1letter.h5'
model_0_oO_name = 'model_0_oO.h5'

characters_folder = 'characters'
test_images_folder = os.path.join(parent_directory, characters_folder)
character_to_test = '{}.png'


In [3]:
from functions_score import *
from functions_char_preparation import *
from char_classification import classification
import shutil
import cv2
import numpy as np
import tensorflow as tf
import string
import pandas as pd
import keras.backend as K #clear RAM
import docx

In [4]:
cwd = os.getcwd()
cwd

'c:\\Users\\JosePombo\\Desktop\\repos\\ocr\\04-Recognition'

In [ ]:
def loadModels():
    characters_all = list(string.printable)[:-6] #+['ç']# <
    j=-1
    dict_target=[]
    for char in characters_all:
        j=j+1
        dict_target.append([char,ord(char),j])
    dictionar=pd.DataFrame(dict_target).rename(columns={0:'Actual_char',1:'Actual_num',2:'Actual_id'})

    dictionar_symbols=dictionar[62:94]
    dictionar_letters=dictionar[0:62]
    #print('dictionar_symbols', dictionar_symbols)
    # print('dictionar_letters', dictionar_letters)


    gan_char_models_path = os.path.join(root, classifiers_folder, gan_char_models_folder, model_digits_letters_name)
    #gan_char_models_path.replace('%s', ord(char))
    for char in characters_all[0:62]:
        print(char)

        #model_temp=tf.keras.models.load_model('C:\\Users\\Administrator\\OCR\\Final\\02-Classifiers\\models_char_gan\\32_\\class_char_model_%s.h5'%(ord(char)))
        gan_char_models_filename = gan_char_models_path.format(ord(char))
        print('gan_char_models_filename', gan_char_models_filename)
        model_temp=tf.keras.models.load_model(gan_char_models_filename)
        #creates variable name dynamically
        exec(f'model_letters{ord(char)} = model_temp')
        K.clear_session()

    mixed_models_path = os.path.join(root, classifiers_folder, mixed_models_folder)
    for char in characters_all[62:94]:
        print(char)
        
        # model_temp=tf.keras.models.load_model('C:\\Users\\Administrator\\OCR\\Final\\02-Classifiers\\\models_char_gan\\32_\\class_char_model_%s.h5'%(ord(char)))
        gan_char_models_filename = gan_char_models_path.format(ord(char))
        print('gan_char_models_filename', gan_char_models_filename)
        model_temp=tf.keras.models.load_model(gan_char_models_filename)
        #creates variable name dynamically
        exec(f'model_symbols{ord(char)} = model_temp')
        K.clear_session()
        
    #model_symbols_letters=tf.keras.models.load_model('C:\\Users\\Administrator\\OCR\\Final\\02-Classifiers\\\models_mixed\\model_0symbol_1letter.h5')
    model_symbols_letters_path = os.path.join(mixed_models_path, model_symbols_letters_name)  
    model_symbols_letters=tf.keras.models.load_model(model_symbols_letters_path)         

    #0 takes 0 ,oO takes 1
    #model_0_oO =tf.keras.models.load_model('C:\\Users\\Administrator\\OCR\\Final\\02-Classifiers\\\models_mixed\\model_0_oO.h5') 
    model_0_oO_path = os.path.join(mixed_models_path, model_0_oO_name)
    model_0_oO =tf.keras.models.load_model(model_0_oO_path)             

    #threshold for _,-
    from pandas import DataFrame
    pd_=DataFrame(np.arange(0,32,1))
    Q1_ = pd_.quantile(0.25)
    Q3_ = pd_.quantile(0.75)
    IQR_ = Q3_ - Q1_



In [8]:
def char_detection(new_img,new_img_normheight,dictionar,Q1_,Q3_): #detect char 
    
    def image_char_prepr(img,dis_bound,blur,gray_thres):
    
        import cv2
        import numpy as np
        import warnings
        warnings.simplefilter('ignore')

        gray_char = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if gray_thres>0:
        #https://stackoverflow.com/questions/49907382/how-to-remove-whitespace-from-an-image-in-opencv
            gray = 255*(gray_char < gray_thres).astype(np.uint8) 
            coords = cv2.findNonZero(gray) # Find all non-zero points (text)
            x, y, w, h = cv2.boundingRect(coords) # Find minimum spanning bounding box
            gray_char = gray_char[y:y+h, x:x+w] # Crop the image - note we do this on the original image

        blur_char = cv2.bilateralFilter(gray_char, blur,blur, blur) #90, 90, 90
        thresh_char = cv2.adaptiveThreshold(blur_char, 255, cv2.ADAPTIVE_THRESH_MEAN_C , 
                                           cv2.THRESH_BINARY_INV, 23, 19)
        constant= cv2.copyMakeBorder(thresh_char.copy(),dis_bound,dis_bound,dis_bound,dis_bound,cv2.BORDER_CONSTANT)

        resized=cv2.resize(constant,(32,32), interpolation = cv2.INTER_NEAREST)
        return resized

    
    import cv2 
    import  numpy as np
    resized=image_char_prepr(new_img,2,20,0)
    resized_normheight=image_char_prepr(new_img_normheight,2,20,0)

    new_image_density=resized.sum()

    x_test_right = np.expand_dims(resized, axis=-1)
    x_test = np.expand_dims(x_test_right, axis=0)
    forecast,dictionartemp = classification(resized,resized_normheight,dictionar,x_test,Q1_,Q3_)
    text_char =str(forecast)
    return text_char   

In [10]:
'''
caracteres com problemas
132
140
147
186
194
233
241
280
288
'''
unhandledChars = [140, 147, 186, 194, 233, 241, 280, 288]

In [12]:
test_char_list = range(1, 560)
for test_char in test_char_list:
    if not test_char  in unhandledChars:
        test_char = str(test_char)
        print('test_char', test_char)
        test_char_image = os.path.join(test_images_folder, character_to_test.format(test_char))
        print('test_char_image', test_char_image)
        img = cv2.imread(test_char_image)
        #print('img', img)
        #img = cv2.imread('C:\\Users\\Administrator\\OCR\\Final\\04-Recognition\\characters\\2.png')

        sorted_ctrs = char_preprocessing_step_1(img)
        #print('sorted_ctrs', sorted_ctrs)
        new_sorted_ctrs = char_preprocessing_step_2(sorted_ctrs,img)
        print('new_sorted_ctrs', new_sorted_ctrs)
        temp_max_yh,temp_min_y = char_preprocessing_step_3(new_sorted_ctrs) #normalize height
        print('temp_max_yh', temp_max_yh)
        print('temp_min_y', temp_min_y)
        char_image,char_image_nh = char_preprocessing_step_4(img,new_sorted_ctrs,temp_max_yh,temp_min_y)

        #print('Q1_', Q1_)
        #print('Q3_', Q3_)
        #print('char_image[0]', char_image)
        #print('char_image_nh[0]', char_image_nh)
        text_char = char_detection(char_image[0],char_image_nh[0],dictionar,Q1_,Q3_)
        #print('text_char', text_char)

test_char 1
test_char_image c:\Users\JosePombo\Desktop\repos\ocr\04-Recognition\characters\1.png
new_sorted_ctrs [[1, 3, 11, 18, 21, 19]]
temp_max_yh 21
temp_min_y 3


TypeError: char_detection() missing 1 required positional argument: 'output'